In [ ]:
import numpy as np
import math

data1 = np.loadtxt("data8(1).txt", delimiter = ' ')
data2 = np.loadtxt("data8(2).txt", delimiter = ' ')

def lloyd(k, e, Y): #Input
    N = Y.shape[1]
    C = Y[:k,[range(0,N)]] #choose randomly centers
    CardY = 1
    
    h = math.inf #initial h0
    p = [] #initial probability
    cov = []
    
    my = 0
    for y in Y:
        my += y / CardY
    
    summ = 0
    for y in Y:
        summ += (y - my) * (y - my).T
    
    for i in range(k):
        p.append(1/k)
        cov.append(1/CardY*summ)
    
    hprev = 0
    while h >= hprev + e:
        n = 0
        
        
        
lloyd(3, 0.0001, data1)
lloyd(3, 0.0001, data2)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn import datasets
import numpy as np
import random
import math

def k_means (data, k):
    instances = data.shape[0]
    labels = np.ones((instances, 1))
    us = assign_centroids(data, k) 
    centroids = np.array(us)
    for i in range(10):
        labels = assignment(data, us, labels, k)
        if empty(labels, k):
            return k_means(data, k)

        us = update(data, labels, us, k)
        centroids = np.vstack((centroids, us))
    return labels


def assign_centroids(data, k):
    attributes = data.shape[1]
    us = np.ones((1, attributes))
    
    for i in range(k):
        index = random.randint(0, len(data) - 1)
        u_new = np.array(data[index])
        us = np.vstack((us, u_new))
        data = np.delete(data, index, 0)
    return us[1:]


def assignment(data, us, labels, k):
    d = lambda p1, p2: math.sqrt(sum((p2 - p1)**2)) # distance function

    for i in range(len(data)):
        point = data[i]
        distances = [d(point, us[j]) for j in range(k)]
        min_distance = min(distances)
        cluster = distances.index(min_distance)
        labels[i] = cluster     
    return labels

def update(data, labels, us, k):
    counters = {} 
    
    for i in range(k):
        counters[i] = 0
    for i in range(len(labels)):
        cluster = int(labels[i])
        us[cluster] = np.add(us[cluster], data[i])
        counters[cluster] += 1
    for i in range(k):
        us[i] = us[i] / counters[i]
    return us

def empty(labels, k):
    for i in range(k):
        count = 0
        for j in range(len(labels)):
            if labels[j] == i:
                count += 1
        if count == 0:
            return True
    return False